In [ ]:
#adding fake news data
import pandas as pd
fake=pd.read_csv(r"C:\Users\Zunnurain.Badar\Fake.csv")
print(fake.head())


In [ ]:
#adding real news data
true=pd.read_csv(r"C:\Users\Zunnurain.Badar\True.csv")
print(true.head())

In [ ]:
print(true.columns)

In [ ]:
#assigning labels and merging both datasets
fake["Label"] =0
true["Label"] =1
ds = pd.concat([fake,true],axis=0)
ds=ds.reset_index(drop=True)
print(ds.shape)

In [ ]:
 ds=ds[[ 'text','Label']]

In [ ]:
print(ds.head())

In [ ]:
#downloading stopwords
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords") 

In [ ]:
stop_words = stopwords.words("english")
print(stop_words)

In [ ]:
print(ds["text"][0])

In [ ]:
import nltk
nltk.download("punkt_tab")


In [ ]:
#tokenizing the datasets
from nltk.tokenize import word_tokenize
sample_text = ds["text"][0]
tokens = word_tokenize(sample_text)
print(tokens)
 

In [ ]:
#removing stopwords
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
print(filtered_tokens)

In [ ]:
cleaned_text= " ".join(filtered_tokens)
print(cleaned_text)

In [ ]:
#doing lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(word,pos="v") for word in filtered_tokens]
 

In [ ]:
print(lemmatized_tokens)

In [ ]:
#converting text into numbers for ml model
from sklearn.feature_extraction.text import TfidfVectorizer
cleaned_articles = ds["text"].tolist()
tfidf = TfidfVectorizer(max_features=5000)  
tfidf_matrix = tfidf.fit_transform(cleaned_articles)
print("TF-IDF Shape:", tfidf_matrix.shape) 

In [ ]:
#splitting the training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_matrix, ds["Label"], test_size=0.2, random_state=42
)
print("Training Set Size:", X_train.shape)
print("Testing Set Size:", X_test.shape)


In [ ]:
#training model on datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
#saving the files
import joblib
joblib.dump(model, "model.pkl")
joblib.dump(tfidf, "vectorizer.pkl")
print("Model & Vectorizer Saved Successfully!")


In [ ]:
!pip install flask


In [ ]:
pip install flask joblib


In [ ]:
#flask for web page
from flask import Flask, request, jsonify, render_template_string
import joblib
from werkzeug.serving import run_simple
import threading
app = Flask(__name__)
model = joblib.load("model.pkl")
vectorizer = joblib.load("vectorizer.pkl")
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Fake News Detection</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin: 50px;
            background-color: #f4f4f4;
        }
        .container {
            width: 50%;
            margin: auto;
            padding: 20px;
            background: white;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            border-radius: 10px;
        }
        textarea {
            width: 80%;
            height: 100px;
            margin: 10px 0;
            padding: 10px;
        }
        button {
            background: darkgreen;
            color: white;
            padding: 10px 20px;
            border: none;
            cursor: pointer;
        }
        button:hover {
            background: darkgreen;
        }
        #result {
            font-size: 18px;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Fake News Detection</h2>
        <textarea id="newsInput" placeholder="Enter your news text here..."></textarea>
        <button id="checkNewsBtn">Check News</button>
        <p id="result"></p>
    </div>

      <script>
    document.getElementById("checkNewsBtn").addEventListener("click", function () {
        let text = document.getElementById("newsInput").value;
        let resultElement = document.getElementById("result");

    
        resultElement.innerText = "Checking...";
        resultElement.style.color = "black"; 

        fetch("/predict", {
            method: "POST",
            headers: {
                "Content-Type": "application/json"
            },
            body: JSON.stringify({ text: text })
        })
        .then(response => response.json())
        .then(data => {
            setTimeout(() => {
                let resultText = data.prediction === 1 ? "📰 Real News" : "⚠️ Fake News";
                resultElement.innerText = resultText;
                resultElement.style.color = data.prediction === 1 ? "green" : "red";

                
                setTimeout(() => {
                    resultElement.innerText = "";
                }, 4000); // 4 seconds (4000 milliseconds)
            }, 100); // Small delay for better UI
        })
        .catch(error => {
            console.error("Error:", error);
            resultElement.innerText = "❌ Error occurred";
            resultElement.style.color = "red";

            
            setTimeout(() => {
                resultElement.innerText = "";
            }, 5000);
        });

        document.getElementById("newsInput").value = ""; 
    });
</script>

</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    text = data.get("text", "")

    text_vectorized = vectorizer.transform([text])

    #getting prediction from model
    prediction = model.predict(text_vectorized)[0]                    
    return jsonify({"prediction": int(prediction)})

# function to run flask on jupyter notebook
def run_flask():
    run_simple('localhost', 5000, app, use_reloader=False)

threading.Thread(target=run_flask, daemon=True).start()
